# -----------------------------------
# 목차
## Step 1. 데이터 다운로드
## Step 2. 데이터 읽어오기
## Step 3. 데이터 정제
## Step 4. 평가 데이터셋 분리
## Step 5. 인공지능 만들기
# -----------------------------------

## Step 1. 데이터 다운로드
### Cloud shell에 아래의 명령어 입력하기
### mkdir -p ~/aiffel/lyricist/models
###  ln -s ~/data ~/aiffel/lyricist/data

## Step 2. 데이터 읽어오기¶

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/Exploration/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## Step 3. 데이터 정제

### shakespeare.txt의 경우, txt파일을 열어보면
### - First Citizen:
### - Before we proceed any further, hear me speak.
### -  
### - All:
### - Speak, speak.
### 이렇게 되어 있으나 lyrics 파일의 경우, 예를 들어 adele.txt 파일을 열어보면
### - Looking for some education
### - Made my way into the night
### - All that bullshit conversation
### - 이렇게 되어 있기 때문에 '문장의 끝이 :'로 끝나거나 '공백'인 문장을 제외할 필요가 없음

In [2]:
import re 
import numpy as np
import tensorflow as tf

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 5. 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [3]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    
    if len((preprocessed_sentence).split()) > 15 : continue
    
    
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [4]:
def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )

    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2971 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  117 ...    0    0    0]
 [   2  258  195 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7ffbfc8db220>


In [5]:
print(tensor[:3, :10])

[[   2   50    4   95  303   62   53    9  946 6269]
 [   2   15 2971  872    5    8   11 5747    6  374]
 [   2   33    7   40   16  164  288   28  333    5]]


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6269    3    0    0    0]
[  50    4   95  303   62   53    9  946 6269    3    0    0    0    0]


# Step 4. 평가 데이터셋 분리

In [8]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=7)
print('enc_train 개수: ', len(enc_train),', dec_train 개수: ', len(dec_train))

enc_train 개수:  124981 , dec_train 개수:  124981


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# Step 5. 인공지능 만들기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 7.4022588e-05,  3.3047496e-04,  4.1235248e-06, ...,
         -2.6664431e-05,  2.4140203e-04, -1.3065625e-04],
        [ 1.4566847e-04,  5.1905907e-04,  1.9473209e-05, ...,
          8.1593535e-05,  5.0338678e-04, -2.9995618e-04],
        [ 3.0166816e-04,  7.3562260e-04,  1.8172688e-04, ...,
          2.7605076e-04,  4.5113818e-04, -3.8082935e-04],
        ...,
        [-4.0743433e-04, -2.6388827e-04, -5.5463483e-05, ...,
          1.0067815e-03, -2.7662553e-04,  8.2461257e-04],
        [-7.1867072e-04, -1.6968697e-04, -2.0632421e-04, ...,
          1.4308587e-03, -7.1830372e-04,  1.1762247e-03],
        [-1.0445340e-03, -6.2130806e-05, -3.4317019e-04, ...,
          1.8271870e-03, -1.1522550e-03,  1.4578409e-03]],

       [[ 7.4022588e-05,  3.3047496e-04,  4.1235248e-06, ...,
         -2.6664431e-05,  2.4140203e-04, -1.3065625e-04],
        [ 1.3286280e-04,  6.6878489e-04,  1.8527733e-04, ...,
         -2.5344722e-04, 

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18466976  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  32780704  
_________________________________________________________________
dense (Dense)                multiple                  24302025  
Total params: 78,621,961
Trainable params: 78,621,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, validation_data=(enc_val,dec_val), batch_size=16)

Epoch 1/10
7812/7812 [==============================] - 604s 77ms/step - loss: 2.9281 - val_loss: 2.6812
Epoch 2/10
7812/7812 [==============================] - 604s 77ms/step - loss: 2.4242 - val_loss: 2.4560
Epoch 3/10
7812/7812 [==============================] - 604s 77ms/step - loss: 2.0202 - val_loss: 2.3355
Epoch 4/10
7812/7812 [==============================] - 604s 77ms/step - loss: 1.6980 - val_loss: 2.3028
Epoch 5/10
 457/7812 [>.............................] - ETA: 8:39 - loss: 1.3650

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

In [ ]:
a=(enc_val,dec_val)
EPOCHS = 10
history = model.fit(enc_train, dec_train,
                    epochs=EPOCHS,
                    validation_data=a)